In [1]:
import os

In [2]:
%pwd

'd:\\kidney_tumor_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\kidney_tumor_classification'

In [5]:
os.chdir("research")

In [6]:
%pwd

'd:\\kidney_tumor_classification\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'd:\\kidney_tumor_classification'

In [10]:
pip install dagshub

  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.0/13.5 MB ? eta -:--:--
   -------- ------------------------------- 2.9/13.5 MB 13.9 MB/s eta 0:00:01
   ------------------- -------------------- 6.6/13.5 MB 15.5 MB/s eta 0:00:01
   ------------------------------ --------- 10.2/13.5 MB 15.9 MB/s eta 0:00:01
   -------------------------------------- - 13.1/13.5 MB 16.4 MB/s eta 0:00:01
   -------------------------------------- - 13.1/13.5 MB 16.4 MB/s eta 0:00:01
   -------------------------------------- - 13.1/13.5 MB 16.4 MB/s eta 0:00:01
   -------------------------------------- - 13.1/13.5 MB 16.4 MB/s eta 0:00:01
   ---------------------------------------  13.4/13.5 MB 8.5 MB/s eta 0:00:01
   ---------------------------------------- 13.5/13.5 MB 7.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ----------------------------------- ---- 3.4/3.8 MB 16.7 MB/s eta 0:00:01
   --

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.2.2 requires Flask<3, but you have flask 3.1.0 which is incompatible.
mlflow 2.2.2 requires packaging<24, but you have packaging 24.2 which is incompatible.
mlflow 2.2.2 requires pytz<2023, but you have pytz 2025.2 which is incompatible.


In [12]:
import dagshub
dagshub.init(repo_owner='YemineniPavan', repo_name='kidney_tumor_classification', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=42ad7817-1942-4449-9ea8-9f1290246575&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=b2c05fa2f49a4ea1377853dbdcaa4fa923592b86d030d916bbcf92e555d9578e




Accessing as YemineniPavan

Initialized MLflow to track repo "YemineniPavan/kidney_tumor_classification"

Repository YemineniPavan/kidney_tumor_classification initialized!

In [13]:
import tensorflow as tf

In [14]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [30]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    mlflow_uri : str
    params_image_size : list
    params_batch_size : int

In [31]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [36]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Kidney ct scan images",
            mlflow_uri="https://dagshub.com/YemineniPavan/kidney_tumor_classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [37]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [38]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [39]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-04-16 11:01:08,842: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-16 11:01:08,842: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-16 11:01:08,842: INFO: common: created directory at: artifacts]
Found 2207 images belonging to 2 classes.
138/138 [==============================] - 383s 3s/step - loss: 24.9962 - accuracy: 0.6901
[2025-04-16 11:07:33,491: INFO: common: json file saved at: scores.json]


2025/04/16 11:07:35 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-04-16 11:07:37,289: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\WELCOME\AppData\Local\Temp\tmp49hu7h_9\model\data\model\assets
[2025-04-16 11:07:38,294: INFO: builder_impl: Assets written to: C:\Users\WELCOME\AppData\Local\Temp\tmp49hu7h_9\model\data\model\assets]


d:\kidney_tumor_classification\kidney\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
Successfully registered model 'VGG16Model'.
2025/04/16 11:08:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
